<a href="https://colab.research.google.com/github/joaopfusco/bible-chatbot/blob/main/BibleChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação da api do google

In [1]:
!pip install -q -U google-generativeai

# Importação das Bibliotecas Necessárias para Utilização do ChatBot

In [2]:
from google.colab import userdata
import google.generativeai as genai
import textwrap
from IPython.display import display, Markdown
import numpy as np
import pandas as pd

# Configurando GenAI com API Key

In [4]:
api_key = userdata.get('API_KEY')
genai.configure(api_key=api_key)

# Função de MarkDown para melhor vizualização

In [5]:
def markdown(text):
  text = text.replace('•', ' *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _ : True))

# Dois Chats

*   Chat 1: utiliza do embedding para consultas mais precisas relacionadas
aos quatro primeiros evangelhos do Novo Testamento (Mateus, Marcos, Lucas e João)
*   Chat 2: conhecimento geral da Bíblia Sagrada para perguntas gerais sobre
o Evangelho



# CHAT 1

# Definição da lista de Documentos sobre os quatro primeiros evangelhos do Novo Testamento

In [6]:
documents = [
  {
    "title": "Evangelho segundo Marcos",
    "content": "O Evangelho segundo Marcos apresenta a história de Jesus Cristo, Filho de Deus, começando com o ministério de João Batista e o batismo de Jesus. Marcos destaca as ações e milagres de Jesus, como curas, exorcismos e a multiplicação dos pães. Ele enfatiza a autoridade de Jesus em seu ensino e seus confrontos com as autoridades religiosas da época. O evangelho culmina com a Paixão, morte e ressurreição de Jesus, enfatizando o significado da sua morte sacrificial e a importância da fé nele."
  },
  {
    "title": "Evangelho segundo Lucas",
    "content": "O Evangelho de Lucas oferece uma narrativa detalhada da vida e ministério de Jesus Cristo. Dirigido a Teófilo, o evangelho traça a linhagem de Jesus, narrando seu nascimento, infância e batismo. Lucas enfatiza a compaixão de Jesus pelos pobres e marginalizados, destacando seus ensinamentos sobre o amor, o perdão e a importância da oração. O evangelho culmina com a entrada triunfal de Jesus em Jerusalém, sua Paixão, morte e ressurreição, culminando com sua ascensão ao céu."
  },
  {
    "title": "Evangelho segundo João",
    "content": "O Evangelho de João apresenta uma perspectiva única sobre Jesus, enfatizando sua divindade como a Palavra de Deus encarnada. O evangelho destaca os \"sinais\" ou milagres de Jesus como demonstrações de sua glória, levando as pessoas à fé. João enfatiza o amor de Jesus por seus discípulos e seus ensinamentos sobre o Espírito Santo como consolador. A narrativa culmina com a Paixão, morte e ressurreição de Jesus, enfatizando o significado de sua morte como sacrifício pela vida do mundo."
  },
  {
    "title": "Evangelho Segundo Mateus",
    "content": "O Evangelho de Mateus apresenta Jesus como o Messias prometido, cumprindo as profecias do Antigo Testamento. O evangelho traça a linhagem de Jesus de volta a Abraão e narra seu nascimento miraculoso, fuga para o Egito e retorno a Nazaré. Mateus enfatiza o ensino de Jesus, incluindo o Sermão da Montanha, e organiza o evangelho em torno de cinco discursos principais. O evangelho culmina com a Paixão, morte e ressurreição de Jesus, enfatizando a importância de sua missão e o estabelecimento de sua Igreja."
  }
]

# Criação do DataFrame e Função de Consulta

In [7]:
def embed(title, text):
  return genai.embed_content(model='models/embedding-001',
                              title=title,
                              content=text,
                              task_type='RETRIEVAL_DOCUMENT')['embedding']

In [8]:
def consult(consulta):
  embedding = genai.embed_content(model='models/embedding-001',
                              content=consulta,
                              task_type='RETRIEVAL_QUERY')['embedding']
  produts = np.dot(np.stack(df['Embeddings']), embedding)
  indice = np.argmax(produts)
  return df.iloc[indice]['Conteúdo']

In [ ]:
df = pd.DataFrame(documents)
df.columns = ['Título', 'Conteúdo']
df['Embeddings'] = df.apply(lambda row: embed(row['Título'], row['Conteúdo']), axis=1)
df

# CHAT 2

# Configurações inicias do modelo

In [ ]:
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
]

system_instruction = """Você é um chat bot usado em ingrejas evangélicas para
responder questionamentos com embasamento bíblico evangélico,
logo suas respostas devem ser baseadas na Bíblia Sagrada Evangélica e
sempre utilizar de versículos para responder.
Você não deve responder a perguntas que não tem relação com a Bíblia."""

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

# Criação do Chat 2

In [ ]:
chat = model.start_chat(history=[])

# UTILIZE OS CHATS!

# Utilize o CHAT 1:

In [ ]:
while True:
  prompt = input("Digite aqui: ").lower().strip()
  print('-' * 150)

  if prompt == "fim":
    break

  response = consult(prompt)

  # user
  display(markdown(f'\n**user**: {prompt}'))
  print('-' * 150)

  # model
  display(markdown(f'\n**model**: {response}'))
  print('-' * 150)

# Utilize o CHAT 2:

In [ ]:
while True:
  prompt = input("Digite aqui: ").lower().strip()
  print('-' * 150)

  if prompt == "fim":
    break

  response = chat.send_message(prompt)

  # user
  user_message = chat.history[-2]
  display(markdown(f'\n**user**: {user_message.parts[0].text}'))
  print('-' * 150)

  # model
  model_message = chat.history[-1]
  display(markdown(f'\n**model**: {model_message.parts[0].text}'))
  print('-' * 150)